In [1]:
import MySQLdb as db
import pandas as pd
from pandas import DataFrame

In [2]:
# connects script to customer aws-hosted mysql db
database = db.connect('ec2-54-242-241-88.compute-1.amazonaws.com','capstone','suite208208','libras')

# stores the location sample results are stored - update as required.
csvfile = 'C:/Users/bjtur/Documents/Capstone/libras_sample.csv'

# stores a common select statement used to extract and append table headers, then the shipment records themselves
select_columns = ("SELECT id, business_sid, industry, sub_industry, shipper, service_type_description, "
                  + "package_count, weight, year_week, shipment_date, delivery_date, "
                  + "delivery_time, freight_charges, freight_discount_amount, misc_charges, "
                  + "misc_discount_amount, net_charge_amount, zone, sender_city, sender_state, sender_zip, "
                  + "sender_country, recipient_city, recipient_state, recipient_zip, recipient_country")

In [5]:
def db_batch (lower, higher):
    """
    Appends a sample of a batch of records to csvfile
    """
    data = pd.read_sql_query(select_columns + " FROM shipment_details limit {}, {};".format(lower, higher), database)
    data_sample = data.sample(frac=0.1, replace=False)
    with open(csvfile, 'a', newline='\n') as f:
        data_sample.to_csv(f, header=False, index=False, sep = ';')

In [6]:
headers = pd.read_sql_query(select_columns + " FROM shipment_details limit 0;", database)
with open(csvfile, 'a', newline='\n') as f:
    headers.to_csv(f, index=False, sep = ';')

In [23]:
import time
start = time.time()
for index in range(0, 30000000, 1000000): # range intentionally extends beyond the known number of records
    batch_start = time.time()
    try:
        db_batch(index, 100000)
        print("Index {} | Total time: {} min | Batch time: {} sec".format(index,
                                                                          int(round((time.time() - start)/60, 2)),
                                                                          int(round(time.time() - batch_start))))
    except:
        print("============================================")
        print("Total time: {} min".format(int(round((time.time() - start)/60, 2))))
        break

Index 0 | Total time: 0 min | Batch time: 8 sec
Index 1000000 | Total time: 0 min | Batch time: 10 sec
Index 2000000 | Total time: 0 min | Batch time: 17 sec
Index 3000000 | Total time: 0 min | Batch time: 15 sec
Index 4000000 | Total time: 1 min | Batch time: 16 sec
Index 5000000 | Total time: 1 min | Batch time: 19 sec
Index 6000000 | Total time: 1 min | Batch time: 20 sec
Index 7000000 | Total time: 2 min | Batch time: 23 sec
Index 8000000 | Total time: 2 min | Batch time: 25 sec
Index 9000000 | Total time: 2 min | Batch time: 26 sec
Index 10000000 | Total time: 3 min | Batch time: 33 sec
Index 11000000 | Total time: 4 min | Batch time: 29 sec
Index 12000000 | Total time: 4 min | Batch time: 33 sec
Index 13000000 | Total time: 5 min | Batch time: 33 sec
Index 14000000 | Total time: 5 min | Batch time: 34 sec
Index 15000000 | Total time: 6 min | Batch time: 37 sec
Index 16000000 | Total time: 6 min | Batch time: 38 sec
Index 17000000 | Total time: 7 min | Batch time: 43 sec
Index 180

In [7]:
result = pd.read_sql_query("SELECT * FROM shipment_details limit 100;", database)

In [8]:
result

,id,year_week,tracking,shipper,business_sid,industry,sub_industry,service_type_description,weight,entered_weight,...,sender_zip,sender_state,sender_country,recipient_address_line,recipient_city,recipient_zip,recipient_state,recipient_country,created_at,updated_at
0,1,201823,780969579159,fedex,185B38ACC0,Other,Other,home delivery,23.0,15.00,...,33016,FL,US,516 REDWOOD AVE,SACRAMENTO,958152423,CA,US,2018-06-04 10:04:06,2018-06-04 14:07:17
1,2,201823,780991368310,fedex,185B38ACC0,Other,Other,home delivery,13.0,12.70,...,33016,FL,US,177 PINE HILL RD,STERLING,063772306,CT,US,2018-06-04 10:04:06,2018-06-04 14:07:18
2,3,201823,414462056260,fedex,E351B44FE9,Other,Other,international priority direct distribution,55.4,55.40,...,24012,VA,US,6895 BRAMALEA RD,MISSISSAUGA,L9Z8Z8,ON,CA,2018-06-04 10:02:34,2018-06-04 10:02:34
3,4,201823,414462054934,fedex,E351B44FE9,Other,Other,international priority direct distribution,56.0,46.50,...,24012,VA,US,6895 BRAMALEA RD,MISSISSAUGA,L9Z8Z8,ON,CA,2018-06-04 10:02:34,2018-06-04 10:02:34
4,5,201823,414462053673,fedex,E351B44FE9,Other,Other,international priority direct distribution,35.2,35.20,...,24012,VA,US,6895 BRAMALEA RD,MISSISSAUGA,L9Z8Z8,ON,CA,2018-06-04 10:02:34,2018-06-04 10:02:34
5,6,201823,417419309869,fedex,WDUUGPLE,Retail,Ecommerce,international priority,633.0,545.50,...,06831,CT,US,350 AVALON DRIVE,LABRADOR CITY,A2V1K6,NF,CA,2018-06-04 09:33:25,2018-06-04 09:33:25
6,7,201823,409867363481,fedex,A9F4274EEB,Retail,Ecommerce,international economy,1129.7,1129.70,...,311400,CN,CN,300 Hatteras Ave,Clermont,34711,FL,US,2018-06-04 10:55:14,2018-06-04 10:55:14
7,8,201823,414462079835,fedex,E351B44FE9,Other,Other,international priority direct distribution,45.7,45.70,...,24012,VA,US,24 AERO DRIVE N.E.,CALGARY,T9Z8Z8,AB,CA,2018-06-04 10:02:34,2018-06-04 10:02:34
8,9,201823,414462077177,fedex,E351B44FE9,Other,Other,international priority direct distribution,31.5,31.50,...,24012,VA,US,24 AERO DRIVE N.E.,CALGARY,T9Z8Z8,AB,CA,2018-06-04 10:02:34,2018-06-04 10:02:34
9,10,201823,414462081170,fedex,E351B44FE9,Other,Other,international priority direct distribution,15.6,15.60,...,24012,VA,US,24 AERO DRIVE N.E.,CALGARY,T9Z8Z8,AB,CA,2018-06-04 10:02:34,2018-06-04 10:02:34
